_한국어로 기계번역됨_


# Pydantic 모델을 그래프 상태로 사용하는 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 친숙함을 가정합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#state">
                    상태
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes">
                    노드
                </a>
            </li>
            <li>
                <a href="https://github.com/pydantic/pydantic">
                    Pydantic
                </a>: 이는 런타임 검증을 위한 인기 있는 파이썬 라이브러리입니다.
            </li>
        </ul>
    </p>
</div>

[A StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph)는 초기화 시 `state_schema` 인자를 받아들여 그래프 내 노드가 접근하고 업데이트할 수 있는 상태의 "형태"를 지정합니다.

우리의 예에서는 일반적으로 `state_schema`에 파이썬 네이티브 `TypedDict`를 사용하지만(또는 [MessageGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#messagegraph)의 경우 [리스트](https://docs.python.org/3/library/stdtypes.html#list)), `state_schema`는 어떤 [유형](https://docs.python.org/3/library/stdtypes.html#type-objects)도 될 수 있습니다.

이 사용 가이드에서는 [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/)이 `state_schema`에 어떻게 사용되어 **입력**에 대한 런타임 검증을 추가할 수 있는지 보여줍니다.


<div class="admonition note">
    <p class="admonition-title">알려진 한계</p>
    <p>
        <ul>
            <li>
              이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하며, <code>langchain-core >= 0.3</code>이 필요합니다. <code>langchain-core < 0.3</code>을 사용하면 Pydantic v1과 v2 <code>BaseModels</code>의 혼합으로 인해 오류가 발생합니다.      
            </li>        
            <li>
                현재 그래프의 `output`은 **Pydantic 모델의 인스턴스가 아닙니다**.
            </li>
            <li>
                런타임 검증은 노드로의 **입력**에 대해서만 발생하며, 출력에는 발생하지 않습니다.
            </li>
            <li>
                Pydantic의 검증 오류 추적은 오류가 발생한 노드를 보여주지 않습니다.
            </li>
        </ul>
    </p>
</div>


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 당신의 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 향상시킬 수 있습니다. LangSmith는 LangGraph로 구축된 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 - 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>에서 확인해보세요.
    </p>
</div>


## 입력 검증


In [4]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

from pydantic import BaseModel


# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    a: str


def node(state: OverallState):
    return {"a": "goodbye"}


# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(node)  # node_1 is the first node
builder.add_edge(START, "node")  # Start the graph with node_1
builder.add_edge("node", END)  # End the graph after node_1
graph = builder.compile()

# Test the graph with a valid input
graph.invoke({"a": "hello"})


{'a': 'goodbye'}

유효하지 않은 입력으로 그래프를 호출하세요.


In [5]:
try:
    graph.invoke({"a": 123})  # Should be a string
except Exception as e:
    print("An exception was raised because `a` is an integer rather than a string.")
    print(e)


An exception was raised because `a` is an integer rather than a string.
1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


## 다중 노드

런타임 검증은 다중 노드 그래프에서도 작동합니다. 아래의 예제에서 `bad_node`는 `a`를 정수로 업데이트합니다.

런타임 검증은 **입력**에서 발생하므로, 검증 오류는 `bad_node`가 상태 업데이트를 반환할 때가 아니라 `ok_node`가 호출될 때 발생합니다.


In [6]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

from pydantic import BaseModel


# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    a: str


def bad_node(state: OverallState):
    return {
        "a": 123  # Invalid
    }


def ok_node(state: OverallState):
    return {"a": "goodbye"}


# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(bad_node)
builder.add_node(ok_node)
builder.add_edge(START, "bad_node")
builder.add_edge("bad_node", "ok_node")
builder.add_edge("ok_node", END)
graph = builder.compile()

# Test the graph with a valid input
try:
    graph.invoke({"a": "hello"})
except Exception as e:
    print("An exception was raised because bad_node sets `a` to an integer.")
    print(e)


An exception was raised because bad_node sets `a` to an integer.
1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
